# ML Pipeline Preparation

In [1]:
!pip install SQLAlchemy
!pip install nltk
!pip install scikit-learn

import nltk

nltk.download(['punkt', 'stopwords', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\netxph\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\netxph\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\netxph\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# import libraries
import pandas as pd
import numpy as np
import re
import joblib

from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/processed/DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.genre.value_counts()

news      13035
direct    10747
social     2394
Name: genre, dtype: int64

In [5]:
GenreType = pd.CategoricalDtype(categories=["news", "direct", "social"], ordered=False)
df.genre = df.genre.astype(GenreType)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26176 entries, 0 to 26175
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   id                      26176 non-null  int64   
 1   message                 26176 non-null  object  
 2   original                10153 non-null  object  
 3   genre                   26176 non-null  category
 4   related                 26176 non-null  int64   
 5   request                 26176 non-null  int64   
 6   offer                   26176 non-null  int64   
 7   aid_related             26176 non-null  int64   
 8   medical_help            26176 non-null  int64   
 9   medical_products        26176 non-null  int64   
 10  search_and_rescue       26176 non-null  int64   
 11  security                26176 non-null  int64   
 12  military                26176 non-null  int64   
 13  water                   26176 non-null  int64   
 14  food                  

In [6]:
X = df["message"]
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [7]:
y = df.drop(columns=["id", "message", "genre", "original"])
# y = df[["storm", "fire", "earthquake", "cold"]]
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [8]:
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words("english")

def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)

    tokens = [lemmatizer.lemmatize(t).strip() for t in tokens if t not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ("count", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(RandomForestClassifier(random_state=42, n_jobs=-1)))])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('count',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000192FF9E9F70>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1,
                                                                        random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_pred = pipeline.predict(X_test)

In [13]:
def get_metrics(y_test, y_pred):
    scores = []

    for i, col in enumerate(y.columns):
        report = classification_report(y_test.iloc[:, i], y_pred[:, i], output_dict=True, zero_division=0)

        scores.append({ 
            "category": col,
            "precision_0": report["0"]["precision"],
            "recall_0": report["0"]["recall"],
            "f1_0": report["0"]["f1-score"],
            "support_0": report["0"]["support"],
            "precision_1": report["1"]["precision"],
            "recall_1": report["1"]["recall"],
            "f1_1": report["1"]["f1-score"],
            "support_1": report["1"]["support"],
            "accuracy": report["accuracy"],
            "precision_macro_avg": report["macro avg"]["precision"],
            "recall_macro_avg": report["macro avg"]["recall"],
            "f1_macro_avg": report["macro avg"]["f1-score"],
            "support_macro_avg": report["macro avg"]["support"],
            "precision_weighted_avg": report["weighted avg"]["precision"],
            "recall_weighted_avg": report["weighted avg"]["recall"],
            "f1_weighted_avg": report["weighted avg"]["f1-score"],
            "support_weighted_avg": report["weighted avg"]["support"]
        })

    return pd.DataFrame.from_records(scores)

In [14]:
get_metrics(y_test, y_pred).mean()

<ipython-input-14-ce8e101c2ec4>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  get_metrics(y_test, y_pred).mean()


precision_0                  0.948083
recall_0                     0.973427
f1_0                         0.958689
support_0                 5957.200000
precision_1                  0.642485
recall_1                     0.213897
f1_1                         0.264486
support_1                  586.800000
accuracy                     0.947669
precision_macro_avg          0.795284
recall_macro_avg             0.593662
f1_macro_avg                 0.611588
support_macro_avg         6544.000000
precision_weighted_avg       0.939202
recall_weighted_avg          0.947669
f1_weighted_avg              0.935643
support_weighted_avg      6544.000000
dtype: float64

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
parameters = {
    "clf__estimator": [
        RandomForestClassifier(random_state=42),
        KNeighborsClassifier(),
        LinearSVC(random_state=42)
    ]
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=3)

In [16]:

cv.fit(X_train, y_train)

print("Best Parameters:", cv.best_params_)

Best Parameters: {'clf__estimator': LinearSVC(random_state=42)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
y_pred = cv.predict(X_test)

get_metrics(y_test, y_pred).accuracy.mean()

0.9476248690185121

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
parameters = {
    "clf":  [ MultiOutputClassifier(LinearSVC(random_state=42)), OneVsRestClassifier(LinearSVC(random_state=42)) ]
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=3)

In [19]:
cv.fit(X_train, y_train)

print("Best Parameters:", cv.best_params_)

Best Parameters: {'clf': MultiOutputClassifier(estimator=LinearSVC(random_state=42))}


In [20]:
y_pred = cv.predict(X_test)

get_metrics(y_test, y_pred).accuracy.mean()

0.9476248690185121

### 9. Export your model as a pickle file

In [21]:
joblib.dump(cv.best_estimator_, "../models/disaster_response_model.pkl")

['../models/disaster_response_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.